In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path='/content/notebooks'

os.symlink('.content/drive/My Drive/Colab Notebooks',my_path)
sys.path.insert(0,my_path)

Mounted at /content/drive


###**count, sum 랭크 변수 만드는 함수, 새분류.pkl 이용**

In [ ]:
def make_rank(): #아래의 count항목을 모두 sum으로 바꿔주면 sum으로 rank화 한 테이블 만들어짐. 
  count_pivot = pd.pivot_table(새분류, columns = '새분류', 
                       index = '고객번호',
                       values='count')
  # null값을 0으로 채우고 int로 바꿔줌
  count_pivot.fillna(0, inplace=True)
  count_pivot = count_pivot.astype('int')
  # 행렬 변환
  count_pivot_t = count_pivot.T

  #칼럼명 str로 바꾸기
  count_pivot_t.columns = count_pivot_t.columns.astype('str')

  #각 고객별로 각 품목에 대한 랭크 메기기
  for i in range(1,len(count_pivot_t.columns)+1):
    count_pivot_t[f'rank_{i}'] = count_pivot_t[f'{i}'].rank(method='min')

  # 기존 고객 번호 칼럼 drop
  a = len(count_pivot_t.columns)/2+1
  a = int(a)
  drop_list = range(1, a)
  drop_list = list(drop_list)
  drop_list = list(map(str, drop_list))

  count_pivot_t.drop(drop_list, axis=1, inplace=True)

  # 행렬 다시 변환
  count_pivot_final = count_pivot_t.T

  # rank매겨진 걸 float -> int로 변환
  for i in count_pivot_final.columns :
    count_pivot_final[i] = count_pivot_final[i].apply(lambda x : int(x))

  # 인덱스 및 칼럼 이름 초기화
  count_pivot_final.index = range(1, len(count_pivot_final.index)+1)
  count_pivot_final.index.name = ''
  count_pivot_final.columns.name = ''

  # pkl 저장
  count_pivot_final.to_pickle('/content/drive/MyDrive/cakd3_1차프로젝트_2조/dataset/count_rank.pkl')

make_rank()

# 데이터 ROAD

In [ ]:
import pandas as pd

x_train = pd.read_pickle('')
x_test = pd.read_pickle('')
y_train = pd.read_pickle('')
y_test = pd.read_pickle('')

# xgboost

In [ ]:
from xgboost import XGBClassifier
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

x_features = x_train
y_label = y_train

xgb_clf = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=3, random_state=10) 

scores = cross_val_score(xgb_clf, x_features, y_label, scoring='accuracy', cv=6)
print('교차 검증별 정확도:', np.round(scores,4))
print('평균 검증 정확도:', np.round(np.mean(scores),4))

# xgboost (K-폴드)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold # train_test_split과 같은 모듈에 있다
import numpy as np

features = train.iloc[:,:-1]
label = train.iloc[:,-1]

evals = [(train, train)]
xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate=0.1, max_depth = 3, random_state=156)

kfold = KFold(n_splits=5)
cv_accuracy = []

n_iter = 0
for train_index,test_index in kfold.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = label.iloc[train_index], label.iloc[test_index]
    
    xgb_wrapper.fit(X_train, y_train)
    # dt_clf.fit(X_train,y_train) # 트레인에는 4개의 그룹이 포함되있다. 5개중 1개는 테스트로 보냈기때문//
    pred = xgb_wrapper.predict(X_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도:{1}, 학습 데이터 크기:{2}, 검증 데이터 크기:{3}'
          .format(n_iter,accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

# LightGBM

In [ ]:
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

model = LGBMClassifier()
kf =StratifiedKFold(n_splits=25,  random_state=1)
all_acc = []
fold_idx = 0

features = X_train
labels = y_train

for train_idx, test_idx in kf.split(features, labels):
    
    train_x, train_y = features.iloc[train_idx], labels.iloc[train_idx]
    test_x, test_y = features.iloc[test_idx], labels.iloc[test_idx]
    

    model.fit(train_x, train_y)
    pred_y = model.predict(test_x)
    acc = accuracy_score(test_y, pred_y)
    
    fold_idx += 1
    
    all_acc.append(acc)

print(f"StratifiedKFold 모두 수행 후 평균 예측도 : {np.mean(all_acc)}")


pred = model.predict(X_test)
print('정확도',accuracy_score(y_test, pred))  
print('정밀도',precision_score(y_test, pred)) 
print('재현율',recall_score(y_test, pred))    
print('f1 score',f1_score(y_test, pred))

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_validate
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

# load sample
dataset=data
x = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

dt_clf = DecisionTreeClassifier(random_state=1)
data_train=x
data_labels=y
dt_clf.fit(data_train, data_labels)
pred = dt_clf.predict(x)

scores = cross_val_score(dt_clf, data_train, data_labels, scoring='accuracy', cv=5)
print('교차 검증별 정확도:', np.round(scores,4))
print('평균 검증 정확도:', np.round(np.mean(scores),4))

In [ ]:
import sklearn.metrics as mt
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

kf =StratifiedKFold(n_splits=5,  random_state=1)
all_acc = []
fold_idx = 0

features = x
labels = y

model = DecisionTreeClassifier(random_state=1)

for train_idx, test_idx in kf.split(features, labels):
    
    X_train, y_train = features.iloc[train_idx], labels.iloc[train_idx]
    X_test, y_test = features.iloc[test_idx], labels.iloc[test_idx]
    
    model.fit(X_train, y_train)
    pred_y = model.predict(X_test)
    acc = accuracy_score(y_test, pred_y)
    
    fold_idx += 1
    
    all_acc.append(acc)

print(f"StratifiedKFold 모두 수행 후 평균 예측도 : {np.mean(all_acc)}")

print('\n')
pred = model.predict(X_test)
print('정확도',accuracy_score(y_test, pred))  
print('정밀도',mt.precision_score(y_test, pred)) 
print('재현율',mt.recall_score(y_test, pred))    
print('f1 score',mt.f1_score(y_test, pred))

In [ ]:
from sklearn.model_selection import GridSearchCV


dt_clf = DecisionTreeClassifier(random_state=33)
parameters = {'criterion': ['gini', 'entropy'],
              'max_depth': [6, 8, 10, 12, 16, 20, 24],
              'min_samples_split': [3, 5],
              'splitter': ['best', 'random']}

grid_dt = GridSearchCV(dt_clf, param_grid = parameters, cv = 5 )

grid_dt.fit(X_train, y_train)

result = pd.DataFrame(grid_dt.cv_results_['params'])
result['mean_test_score'] = grid_dt.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)

In [ ]:
# 결과대입 시각화
dt_clf = DecisionTreeClassifier(criterion='entropy', max_depth=7, splitter = 'best')
dt_clf.fit(x_train,y_train) 

plt.figure(figsize = (20,15))
tree.plot_tree(dt_clf,filled=True,
              rounded=True,fontsize=14)
plt.show()

In [ ]:
feature_importance_values = best_df_clf.feature_importances_
# Top 중요도로 정렬하고, 쉽게 시각화하기 위해 Series 변환
feature_importances = pd.Series(feature_importance_values, index=x_train.columns)
# 중요도값 순으로 Series를 정렬
feature_top20 = feature_importances.sort_values(ascending=False)[:20]
feature_top20

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_clf = RandomForestClassifier(random_state=1)
rf_clf.fit(x_train, y_train)
pred = rf_clf.predict(x_test)
accuracy = accuracy_score(y_test,pred)
print('랜덤포레스트 정확도 : {0:.4f}'.format(accuracy))

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100,200,300],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv=2, n_jobs=-1)
grid_cv.fit(x_train, y_train)

print('최적 하이퍼 파라미터 :\n', grid_cv.best_params_)
print('최고 예측 정확도 :{0:.4f}'.format(grid_cv.best_score_))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import KFold

kf =StratifiedKFold(n_splits=25,  random_state=1)
all_acc = []
fold_idx = 0

features = x_train
labels = y_train

for train_idx, test_idx in kf.split(features, labels):
    
    x_train, y_train = features.iloc[train_idx], labels.iloc[train_idx]
    x_test, y_test = features.iloc[test_idx], labels.iloc[test_idx]
    
    model = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_leaf=8, min_samples_split=20, random_state=1)
    model.fit(x_train,y_train)
    pred_y = model.predict(x_test)
    acc = accuracy_score(y_test, pred_y)
    
    fold_idx += 1
    
    all_acc.append(acc)

print(f"StratifiedKFold 모두 수행 후 평균 예측도 : {np.mean(all_acc)}")


pred = model.predict(x_test)
print('정확도',accuracy_score(y_test, pred))  
print('정밀도',precision_score(y_test, pred)) 
print('재현율',recall_score(y_test, pred))    
print('f1 score',f1_score(y_test, pred))

In [ ]:
ftr_importances_values = model.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=x_train.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
ftr_top20

# Catboost 

In [ ]:
import pickle
with open("/content/drive/MyDrive/cakd3_1차프로젝트_2조/dataset/corr_semi_final.pkl", "rb") as fh: data = pickle.load(fh)
import pickle
with open("/content/drive/MyDrive/cakd3_1차프로젝트_2조/dataset/corr_semi_final_test.pkl", "rb") as fh: test = pickle.load(fh)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from catboost import CatBoostClassifier, *
import pandas as pd
import numpy as np

X = data.iloc[:,:-1]
y = data['증감여부']
X_test = test.iloc[:,:-1]
y_test = test['증감']

In [ ]:
kf =StratifiedKFold(n_splits=25,  random_state=1)
all_acc = []
fold_idx = 0

features = X
labels = y

for train_idx, test_idx in kf.split(features, labels):
    
    train_x, train_y = features.iloc[train_idx], labels.iloc[train_idx]
    test_x, test_y = features.iloc[test_idx], labels.iloc[test_idx]
    
    model = CatBoostClassifier(
        custom_loss=[metrics.Accuracy()],
        random_seed=42,
        logging_level='Silent')
    model.fit(train_x, train_y)
    pred_y = model.predict(test_x)
    acc = accuracy_score(test_y, pred_y)
    
    fold_idx += 1
    
    all_acc.append(acc)

print(f"StratifiedKFold 모두 수행 후 평균 예측도 : {np.mean(all_acc)}")


pred = model.predict(X_test)
print('정확도',accuracy_score(y_test, pred))  
print('정밀도',precision_score(y_test, pred)) 
print('재현율',recall_score(y_test, pred))    
print('f1 score',f1_score(y_test, pred))